In [19]:
import math
import numpy  as np
import pandas as pd
import random
import pickle
import warnings
import inflection
import seaborn as sns

from scipy                 import stats  as ss
from matplotlib            import pyplot as plt
from IPython.display       import Image
from IPython.core.display  import HTML


from sklearn.metrics       import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.ensemble      import RandomForestRegressor
from sklearn.linear_model  import LinearRegression, Lasso
from sklearn.preprocessing import RobustScaler, MinMaxScaler, LabelEncoder
from src.tools             import ml_error

warnings.filterwarnings( 'ignore' )

In [34]:
# Lendo o split gerado na etapa 6
X_train = pd.read_csv('data/X_train.csv')
y_train = pd.read_csv('data/y_train.csv')

X_test = pd.read_csv('data/X_test.csv')
y_test = pd.read_csv('data/y_test.csv')

## 6.3 Manual Feature Selection 

In [7]:
cols_select = ['store',
                'promo',
                'store_type',
                'competition_distance',
                'competition_open_since_month',
                'competition_open_since_year',
                'promo2_since_week',
                'competition_time_month',
                'promo_time_week',
                'day_of_week_sin',
                'day_of_week_cos',
                'month_cos',
                'month_sin',
                'day_sin',
                'day_cos']

## 07 - MACHINE LEARNING MODELLING

In [35]:
X_train = X_train[cols_select]
X_test = X_test[cols_select]

### 7.1 Avarage Model

In [36]:
aux1 = X_test.copy()
aux1['sales'] = y_test.copy()
aux1
#prediction
aux2 = aux1[['store', 'sales']].groupby('store').mean().reset_index().rename(columns={'sales': 'predictions'})
aux1 = pd.merge(aux1, aux2, how = 'left', on = 'store')

yhat_baseline = aux1['predictions']

#performance

baseline_result = ml_error(' Average Model', np.expm1(y_test), np.expm1(yhat_baseline))
baseline_result

,Model Name,MAE,MAPE,RMSE
0,Average Model,1354.800353,0.2064,1835.135542


In [37]:
# Teste com a média do passado e prevendo o futuro

aux1 = X_train.copy()
aux1['sales'] = y_train.copy()
aux1
#prediction
aux2 = aux1[['store', 'sales']].groupby('store').mean().reset_index().rename(columns={'sales': 'predictions'})
aux1 = pd.merge(X_test, aux2, how = 'left', on = 'store')

yhat_baseline = aux1['predictions']

#performance

baseline_result = ml_error(' Average Model', np.expm1(y_test), np.expm1(yhat_baseline))
baseline_result

,Model Name,MAE,MAPE,RMSE
0,Average Model,1429.763326,0.216814,1939.32873


### 7.2. Linear Regression Model

In [38]:
# model
lr = LinearRegression().fit(X_train, y_train)

# prediction
yhat_lr = lr.predict(X_test)
# performance
lr_result = ml_error('Linear Regression', np.expm1(y_test), np.expm1(yhat_lr))
lr_result

,Model Name,MAE,MAPE,RMSE
0,Linear Regression,1906.89389,0.300229,2721.825635


### 7.3. Linear Regression Regularized Model - Lasso


In [39]:
# Model
lrr = Lasso(alpha= 0.001).fit(X_train, y_train)

# Prediction
yhat_lrr = lrr.predict(X_test)
#Performance
lrr_result = ml_error('Linear Regression - Lasso', np.expm1(y_test), np.expm1(yhat_lrr))
lrr_result

,Model Name,MAE,MAPE,RMSE
0,Linear Regression - Lasso,1906.379257,0.295598,2741.81977
